In [1]:
from layer import *

from __future__ import division
import random
import math
import numpy as np
import pandas as pd
import time

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(load_iris().data)
y = np.vectorize(lambda x: 1 if(x>0) else 0)(load_iris().target).astype(np.float64)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state=10, stratify=y)
X_train = np.transpose(X_train)
X_test = np.transpose(X_test)
y_train = y_train.reshape(1,-1)
y_test = y_test.reshape(1,-1)

In [2]:
global global_best_position
global global_best_cost

class Particle:
    global input_size
    global computational_layer
    global ouput_layer
    
    def __init__(self, dimensions):
        self.position = np.random.random((dimensions,))
        self.dimensions = dimensions
        self.position_bounds = [-10,10]
        self.velocity_bounds = [(self.position_bounds[1]-self.position_bounds[0])/2, -1 * self.position_bounds[1]]
        self.velocity = np.random.random((dimensions,))
        self.best_position = np.random.random((dimensions,))
        self.best_cost, _ = forward_pass_weights_bin(self.best_position,X_train, y_train ,input_size, computational_layer, output_layer)
        
        self.c = [2.05,2.05]
        self.learning_rate = 0.8
        
    def evaluate(self, cost):
        if(cost<self.best_cost):
            self.best_cost = cost
            self.best_position = self.position
    
    def update_velocity(self, swarm_best_position, omega):
        self.velocity = omega * (self.velocity) + (self.c[0]*(2.0*random.random()-1.0)*(self.best_position - self.position)) + (self.c[1]*(2.0*random.random()-1.0)*(swarm_best_position - self.position))
        
    def update_position(self):
        self.position = self.position + self.velocity
        self.position = np.vectorize(lambda z: self.position_bounds[1] if z > self.position_bounds[1] else float(z)) (self.position)
        self.position = np.vectorize(lambda z: self.position_bounds[0] if z < self.position_bounds[0] else float(z)) (self.position)
        


In [3]:
class Swarm:
    global input_size
    global computational_layer
    global output_layer
    
    def __init__(self, dimensions, n_population):
        self.particles = [Particle(dimensions) for x in range(n_population)]
        self.swarm_best_particle = self.particles[0]
        self.swarm_best_position = self.particles[0].position
        self.swarm_best_cost, _ = forward_pass_weights_bin(self.swarm_best_position,X_train, y_train ,input_size, computational_layer, output_layer)
    
    def run_n_iterations(self, n_iterations):
        for iteration in range(n_iterations+1):
            omega = float(1.0-(iteration/(n_iterations+1)))
            
            for particle in self.particles:
                particle.update_velocity(self.swarm_best_position, omega)
                particle.update_position()
            
            for particle in self.particles:
                
                cost, _ = forward_pass_weights_bin(particle.position,X_train, y_train ,input_size, computational_layer, output_layer)
                
                particle.evaluate(cost)
                if(cost<self.swarm_best_cost):
                    self.swarm_best_cost = cost
                    self.swarm_best_position = particle.position
                    self.swarm_best_particle = particle

In [4]:
input_size = 4
computational_layer = 5
output_layer = 1
dimensions = calculate_dimensions(input_size, computational_layer, output_layer)

n_particles = 80
n_swarms = 10
dimensions = calculate_dimensions(input_size, computational_layer, output_layer)
l1 = 150
l2 = 300

subswarms = [Swarm(dimensions, n_particles) for x in range(n_swarms)]
collection_best = Swarm(dimensions, n_swarms)

epochs = 1


In [5]:
start = time.time()

for epoch in range(epochs):
    for subswarm in subswarms:
        subswarm.run_n_iterations(l1)

    collection_best.particles = [subswarm.swarm_best_particle for subswarm in subswarms]
    collection_best.run_n_iterations(l2)

end = time.time()
print(end - start)

cost, output = forward_pass_weights_bin(collection_best.swarm_best_position,X_train,y_train,input_size, computational_layer, output_layer)
print(cost)
print(reg_cost(y_train, output))
print(mae(y_train, output))

/Users/sampathroutu/Desktop/research/stochastic-optimization-research/GSO/layer.py:7: RuntimeWarning: overflow encountered in exp
  sigmoid = lambda z: float(1/(1 + np.exp(-z)))


61.97057390213013
0.43989018519879686
9.390168136914067e-307
9.4567527583476e-155


In [6]:
cost, output = forward_pass_weights_bin(collection_best.swarm_best_position,X_test,y_test,input_size, computational_layer, output_layer)
print(cost)
print(reg_cost(y_test, output))
print(mae(y_test, output))



0.43989018519879713
0.0
1.7585295260270084e-287


In [7]:
cost, output = forward_pass_weights_bin(collection_best.swarm_best_position,X_test,y_test,input_size, computational_layer, output_layer)
cost

0.43989018519879713